## Preambule

In [112]:
# General packages
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import xarray as xr
import json

# Plotting
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import n_colors

## Parameter

In [113]:
hist_startyear = 1990

## Paths

In [114]:
path_main = Path("X:/user/dekkerm/Projects/ECEMF_T5.2/")
path_data = path_main / "Data" / str(hist_startyear)
path_figs = path_main / "Figures"

## Read data files

In [115]:
xr_total = xr.open_dataset(path_data / "xr_total.nc")
all_regions_iso = np.load(path_data / "all_regions.npy")
all_regions_names = np.load(path_data / "all_regions_names.npy")
all_countries_iso = np.load(path_data / "all_countries.npy", allow_pickle=True)
all_countries_names = np.load(path_data / "all_countries_names.npy", allow_pickle=True)
only_regions_iso = ['CVF', 'G20', "EU", "G7", "NA", "AU", "AF", "SIDS", "LDC", "WORLD"]
only_regions_names = ['Climate Vulnerability Forum','G20',"European Union","G7","Northern America","Australasia","African Group","Small Island Developing States","Least Developed Countries"]

In [116]:
xr_budgets_static = xr.open_dataset(path_data / "xr_budgets_static.nc")
#xr_budgets_linear = xr.open_dataset(path_data / "xr_budgets_linear.nc")
xr_budgets_scenario = xr.open_dataset(path_data / "xr_budgets_scenario.nc")
df_michel = pd.read_excel(Path("X:/user/dekkerm/Data/") / "NDC" / "NDC_comparison.xlsx", sheet_name="Michel")

In [117]:
df_g = pd.read_excel(Path("X:/user/dekkerm/Data/") / "UNFCCC_Parties_Groups_noeu.xlsx", sheet_name = "Country groups")
countries_iso = np.array(df_g["Country ISO Code"])
countries_name = np.array(df_g["Name"])
countries_iso = np.array(df_g["Country ISO Code"])
group_cvf = countries_iso[np.array(df_g["CVF (24/10/22)"]) == 1]
group_g20 = countries_iso[np.array(df_g["G20"]) == 1]
group_eu = countries_iso[np.array(df_g["EU"]) == 1]
group_g7 = countries_iso[np.array(df_g["G7"]) == 1]
group_na = countries_iso[np.array(df_g["Northern America"]) == 1]
group_um = countries_iso[np.array(df_g["Umbrella"]) == 1]
group_au = countries_iso[np.array(df_g["Australasia"]) == 1]
group_af = countries_iso[np.array(df_g["African Group"]) == 1]
group_sids = countries_iso[np.array(df_g["SIDS"]) == 1]
group_ldc = countries_iso[np.array(df_g["LDC"]) == 1]
group_eg = countries_iso[np.array(df_g["European Group"]) == 1]
group_world = np.copy(countries_iso)

In [118]:
R = [["GF", "Grandfathering", "navy"],
     ["PC", "Per Capita (immediate)", "forestgreen"],
     ["PCC", "Per Capita Convergence", "red"],
     ["F1g", "Approach 1 - GDP (net)", "orange"],
     ["F1h", "Approach 1 - GDP (HDI)", "purple"],
     ["F2", "Approach 2 - GDP (net)", "silver"],
     ["F2C", "Approach 2 - transition", "goldenrod"],
     ]
R = np.array(R).T
colors = R[2]
rules = R[0]
rulenames = R[1]

In [119]:
categories = ['C1+C2', 'C3', 'C6', 'C7']

In [ ]:
var = 'GDR'
reg = 'USA'
for cat_i, cat in enumerate(categories):
    print(cat, float(xr_budgets_static.sel(ISO=reg, Category=cat)[var]))
    print(cat+'T', float(np.sum(xr_budgets_scenario.sel(ISO=reg, Category=cat)[var])))

In [ ]:
var = 'AP'
reg = 'USA'
for cat_i, cat in enumerate(categories):
    print(cat, float(xr_budgets_static.sel(ISO=reg, Category=cat)[var]))
    print(cat+'T', float(np.sum(xr_budgets_scenario.sel(ISO=reg, Category=cat)[var])))

## Some functions

In [122]:
def fill_scenemissions(spend_country, spend_world):
    if not np.isnan(np.nansum(spend_country)) and np.isnan(np.sum(spend_country)):
        spends_new = np.zeros(len(categories))
        frac = np.nanmean(spend_country/spend_world)
        for c in range(len(categories)):
            if np.isnan(spend_country[c]):
                spends_new[c] = frac*spend_world[c]
            else:
                spends_new[c] = spend_country[c]
    else:
        spends_new = spend_country
    return spends_new
categories
spendsw = []
for c in range(len(categories)):
    spent = float(xr_total.sel(Category=categories[c], Time=2030).GHG_f)
    spendsw.append(spent)
spendsw = np.array(spendsw)

In [ ]:
ar_gig = np.zeros(shape=(len(categories), len(rules)+5, len(all_regions_iso)))+np.nan

for c in tqdm(range(len(categories))):
    ccat = categories[c]
    for cty_i, cty in enumerate(all_regions_iso):
        ar_gig[c, 0, cty_i] = float(xr_total.sel(Time=2019, ISO=cty).GHG_p)

        # # Ratios
        # if total_kyoto_2020 > 0:
        #     ratio = total_co2_2020/total_kyoto_2020
        #     if cty == "CHN":
        #         ratio = 1
        #     ar_gig[c, 1, cty_i] = ratio
        
        # Curpol
        df = df_michel[df_michel.ISO == cty]['curpol_2030']
        if len(df) > 0:
            em_curpol = float(df_michel[df_michel.ISO == cty]['curpol_2030'])/1e3
            ar_gig[c, 2, cty_i] = em_curpol

        # NDC
        ndc_malte = [float(xr_total.sel(ISO=cty, Time=2030).NDC_l), float(xr_total.sel(ISO=cty, Time=2030).NDC_h)]
        ar_gig[c, 3, cty_i] = np.nanmean(ndc_malte)
        
        # Effort sharing rules
        for es_i, es in enumerate(rules):
            try:
                ar_gig[c, es_i+4, cty_i] = float(xr_budgets_scenario.sel(ISO=cty, Time=2030, Category=ccat)[es])
            except:
                ar_gig[c, es_i+4,cty_i] = np.nan
        
        # Scenarios
        # spendsw = []
        # for c in range(8):
        #     spent = float(xr_total.sel(Category="C"+str(c+1), Time=2030).GHG_f)
        #     spendsw.append(spent)
        # spendsw = np.array(spendsw)
        # spends = np.zeros(8)
        # for k in range(8):
        #     try:
        #         spends[k] = float(xr_ar6_iso3.sel(Region=cty, Variable='Emissions|Kyoto Gases|w/o LULUCF', Category="C"+str(k+1), Time=2030).Mean/1e3)
        #     except:
        #         spends[k] = np.nan
        # spends_new = fill_scenemissions(spends, spendsw)
        ar_gig[c, len(rules)+4, cty_i] = np.nan#float(xr_total.sel(Time=2030, ISO=cty, Category=ccat).GHG_f)
xr_preprocess = xr.DataArray(data=ar_gig*1e3,
                            dims=["Ccat", "Variable", "Region"],
                            coords=dict(Ccat=categories,
                                        Variable=["Emissions", "Ratio", "CurPol", "NDC"]+list(rules)+["Scenario"],
                                        Region=all_regions_iso))

In [124]:
ar_gig = np.zeros(shape=(len(categories), len(rules)+1, len(np.arange(2020, 2101)), len(all_regions_iso)))+np.nan

for y_i, y in enumerate(np.arange(2020, 2101)):
    for c in range(len(categories)):
        ccat = categories[c]
        #for cty_i, cty in enumerate(all_regions):    
        ar_gig[c, :len(rules), y_i, :] = np.array([xr_budgets_scenario.sel(ISO=all_regions_iso, Time=y, Category=ccat)[i] for i in rules])/1e3
xr_preprocess_temp = xr.DataArray(data=ar_gig*1e3,
                            dims=["Ccat", "Variable", "Time", "Region"],
                            coords=dict(Ccat=categories,
                                        Variable=list(rules) + ['Scenario'],
                                        Time=np.arange(2020, 2101),
                                        Region=all_regions_iso))

# NEW FIGURES

#### Fig. 1 Per-country dashboard (barplots and timeline)

In [125]:
R = [["GF", "Grandfathering", "navy"],
     #["PC", "Per Capita (immediate)", "forestgreen"],
     #["PCC", "Per Capita Convergence", "red"],
     ["F1g", "Approach 1 - GDP (net)", "orange"],
     ["F1h", "Approach 1 - GDP (HDI)", "purple"],
     ["F2", "Approach 2 - GDP (net)", "forestgreen"],
     ["F2C", "Approach 2 - transition", "tomato"],
     ]
R = np.array(R).T
colors = R[2]
rules = R[0]
rulenames = R[1]

Rsub = [["GF", "Grandfathering", "navy"],
     #["PC", "Per Capita (immediate)", "forestgreen"],
     #["PCC", "Per Capita Convergence", "red"],
     ["F1g", "Approach 1 - GDP (net)", "orange"],
     ["F1h", "Approach 1 - GDP (HDI)", "purple"],
     ["F2", "Approach 2 - GDP (net)", "forestgreen"],
     ["F2C", "Approach 2 - transition", "tomato"],
     ]
Rsub = np.array(Rsub).T
colors_sub = Rsub[2]
rules_sub = Rsub[0]
rulenames_sub = Rsub[1]

In [ ]:
fig = make_subplots(rows=2, cols=2,
                    subplot_titles = ("Emissions in 2030",
                                      "Temporal evolution (C1+C2)",
                                      "Total budgets over 2020-2100",
                                      ""),
                    specs=[[{}, {"rowspan": 2}],
                    [{}, None]],
                    horizontal_spacing = 0.06, vertical_spacing=0.06, column_widths=[0.5, 0.5], shared_xaxes=True)

start_country = "USA"

# BAR PART
row, col = (1, 1)
for cat_i, cat in enumerate(categories):
    arrs = np.array(xr_preprocess.sel(Ccat = cat, Variable = ["CurPol", "NDC"]+list(rules)+["Scenario"], Region="USA"))
    fig.add_trace(go.Bar(x=["NDC"]+list(rules),
                        y=arrs[1:-1],
                        name="",
                        hovertemplate="%{x}: %{y}<br>",
                        marker_color=['black']+list(colors),#['skyblue', 'navy', 'forestgreen', 'red', 'red', 'red', 'orange', 'orange','orange', 'silver', 'silver', 'silver', 'violet']+px.colors.sequential.deep[:8],
                        showlegend=False,
                        text=arrs[1:-1].round(0),
                        textposition="outside"), row, col)

# BAR PART
row, col = (2, 1)
for cat_i, cat in enumerate(categories):
    arrs = np.array([xr_budgets_static.sel(Category = cat, ISO="USA")[i] for i in rules])
    #gf, pc, pcc, f1_net, f1_neg, f1_pos, hdi_net, hdi_neg, hdi_pos, f2_net, f2_neg, f2_pos, f2_t = np.array(xr_budgets_static.sel(Region="USA", Ccat = cat, Variable = ["GF", "PC", "PCC", "A1_gdp_net", "A1_gdp_neg", "A1_gdp_pos", "A1_hdi_net", "A1_hdi_neg", "A1_hdi_pos", "A2_gdp_net", "A2_gdp_neg", "A2_gdp_pos", "A2_trans"]).Value)
    fig.add_trace(go.Bar(x=rules, # Cost-optimal later toevoegen
                        y=arrs,
                        name="",
                        hovertemplate="%{x}: %{y}<br>",
                        marker_color=colors,#['navy', 'forestgreen', 'red', 'red', 'red', 'orange', 'orange','orange', 'silver', 'silver', 'silver', 'violet']+px.colors.sequential.deep[:8],
                        showlegend=False,
                        text=arrs.round(0),
                        textposition="outside"), row, col)

# # TEMP PART
row, col = (1, 2)
fig.add_trace(go.Scatter(x=np.arange(1990, 2022),
                        y=list(1e3*np.array(xr_total.GHG_p.sel(Time=np.arange(1990, 2020), ISO=start_country)))+[1e3*float(xr_budgets_scenario.GF.sel(Time=2021, ISO=start_country).mean(dim='Category'))],
                        name='Historical emissions',
                        mode='lines',
                        showlegend=True,
                        line=dict(width=4, color='black')), row, col)
fig.add_trace(go.Scatter(x=[2030, 2030],
                        y=[1e3*xr_total.NDC_l.sel(ISO=start_country, Time=2030), 1e3*xr_total.NDC_h.sel(ISO=start_country, Time=2030)],
                        name='NDC',
                        mode='lines+markers',
                        marker_symbol='diamond-wide',
                        marker_size=15,
                        showlegend=True,
                        line=dict(width=4, color='black')), row, col)
for es_i, es in enumerate(rules_sub):
    fig.add_trace(go.Scatter(x=xr_preprocess_temp.Time,
                            y=np.array(xr_preprocess_temp.sel(Region=start_country, Ccat="C1+C2", Variable=es))*1e3,
                            name=rulenames_sub[es_i],
                            mode='lines',
                            showlegend=True,
                            line=dict(width=4, color=colors_sub[es_i])), row, col)

# DROPDOWN
fig.update_layout(
    updatemenus=[
        {'x':-0.05,
         'xanchor':"right",
         'yanchor':"top",
         'y':1.1,
         'direction':"down",
         'active':int(np.where(all_regions_iso == "USA")[0][0]),
         'showactive':True,
         "buttons": [dict(
                    label=all_regions_names[all_regions_iso == cty][0],#np.array(df_regions[df_regions.ISO == cty].name)[0],
                    method="update",
                    args=[{"y": [np.array(xr_preprocess.sel(Ccat = "C1+C2", Variable = ["NDC"]+list(rules), Region=cty)),
                                 np.array(xr_preprocess.sel(Ccat = "C3", Variable = ["NDC"]+list(rules), Region=cty)),
                                 np.array(xr_preprocess.sel(Ccat = "C6", Variable = ["NDC"]+list(rules), Region=cty)),
                                 np.array(xr_preprocess.sel(Ccat = "C7", Variable = ["NDC"]+list(rules), Region=cty)),
                                 
                                 np.array([xr_budgets_static.sel(ISO=cty, Category = "C1+C2")[i] for i in rules]),
                                 np.array([xr_budgets_static.sel(ISO=cty, Category = "C3")[i] for i in rules]),
                                 np.array([xr_budgets_static.sel(ISO=cty, Category = "C6")[i] for i in rules]),
                                 np.array([xr_budgets_static.sel(ISO=cty, Category = "C7")[i] for i in rules]),
                                list(1e3*np.array(xr_total.GHG_p.sel(Time=np.arange(1990, 2020), ISO=cty)))+[1e3*float(xr_budgets_scenario.GF.sel(Time=2021, ISO=cty).mean(dim='Category'))],
                                np.array([1e3*xr_total.NDC_l.sel(ISO=cty, Time=2030), 1e3*xr_total.NDC_h.sel(ISO=cty, Time=2030)])]+[np.array(xr_preprocess_temp.sel(Region=cty, Ccat="C1+C2", Variable=i))*1e3 for i in rules],

                            "text": [np.array(xr_preprocess.sel(Ccat = "C1+C2", Variable = ["NDC"]+list(rules), Region=cty)).round(0),
                                     np.array(xr_preprocess.sel(Ccat = "C3", Variable = ["NDC"]+list(rules), Region=cty)).round(0),
                                     np.array(xr_preprocess.sel(Ccat = "C6", Variable = ["NDC"]+list(rules), Region=cty)).round(0),
                                     np.array(xr_preprocess.sel(Ccat = "C7", Variable = ["NDC"]+list(rules), Region=cty)).round(0),
                                
                                     np.array([xr_budgets_static.sel(ISO=cty, Category = "C1+C2")[i] for i in rules]).round(0),
                                     np.array([xr_budgets_static.sel(ISO=cty, Category = "C3")[i] for i in rules]).round(0),
                                     np.array([xr_budgets_static.sel(ISO=cty, Category = "C6")[i] for i in rules]).round(0),
                                     np.array([xr_budgets_static.sel(ISO=cty, Category = "C7")[i] for i in rules]).round(0),
                               list(1e3*np.array(xr_total.GHG_p.sel(Time=np.arange(1990, 2020), ISO=cty)))+[1e3*float(xr_budgets_scenario.GF.sel(Time=2021, ISO=start_country).mean(dim='Category'))],
                                  
                                  np.array([1e3*xr_total.NDC_l.sel(ISO=cty, Time=2030), 1e3*xr_total.NDC_h.sel(ISO=cty, Time=2030)])]+[np.array(xr_preprocess_temp.sel(Region=cty, Ccat="C1+C2", Variable=i))*1e3 for i in rules]
                            }],
                ) for cty_i, cty in enumerate(all_regions_iso)]
        },
    ]
)

# TITLE
fig.update_xaxes(title="Time (years)", row=1, col=2)
fig.update_yaxes(title="GHG emissions in 2030<br>(kt CO2e)", row=1, col=1)
fig.update_yaxes(title="Total GHG budget over 2020-2100<br>(Mt CO2e)", row=2, col=1)
fig.update_yaxes(title="GHG emissions (kt CO2e)", row=1, col=2)
fig.update_layout(legend=dict(
    #yanchor="top",
    #y=-0.15,
    x=-0.1,
    y=0.9,
    xanchor="right",
    #x=0.07,
    #orientation="h"
))
fig.update_layout(height=800, autosize=True)

# ROUNDUP
fig.show()
fig_dropdown_country = fig
#py.plot(fig_dropdown_country, filename = 'Budgets_singlecountry_dropdown', auto_open=False, autosize=True)

#### Fig. 1b Lines to check monotomy

In [ ]:
fig = make_subplots(rows=2, cols=3,
                    subplot_titles = ("Approach 1 (GDP)",
                                      "Approach 1 (HDI)",
                                      "Approach 2",
                                      "", "", ""),
                    specs=[[{"rowspan": 2}, {"rowspan": 2}, {"rowspan": 2}],
                    [None, None, None]],
                    horizontal_spacing = 0.06, vertical_spacing=0.06, column_widths=[0.3, 0.3, 0.3], shared_xaxes=True)

start_country = "USA"

cols = [px.colors.sequential.Plasma_r[i*2] for i in range(len(categories))]
# Approach 1 - GDP
row, col = (1, 1)
for cat_i, cat in enumerate(categories):
    fig.add_trace(go.Scatter(x=xr_preprocess_temp.Time,
                            y=np.array(xr_preprocess_temp.sel(Region=start_country, Ccat=cat, Variable="F1g"))*1e3,
                            name=cat,
                            mode='lines',
                            showlegend=True,
                            line=dict(width=4, color=cols[cat_i])), row, col)

# Approach 1 - HDI
row, col = (1, 2)
for cat_i, cat in enumerate(categories):
    fig.add_trace(go.Scatter(x=xr_preprocess_temp.Time,
                            y=np.array(xr_preprocess_temp.sel(Region=start_country, Ccat=cat, Variable="F1h"))*1e3,
                            name=cat,
                            mode='lines',
                            showlegend=False,
                            line=dict(width=4, color=cols[cat_i])), row, col)

# Approach 2 - GDP
row, col = (1, 3)
for cat_i, cat in enumerate(categories):
    fig.add_trace(go.Scatter(x=xr_preprocess_temp.Time,
                            y=np.array(xr_preprocess_temp.sel(Region=start_country, Ccat=cat, Variable="F2"))*1e3,
                            name=cat,
                            mode='lines',
                            showlegend=False,
                            line=dict(width=4, color=cols[cat_i])), row, col)

# DROPDOWN
fig.update_layout(
    updatemenus=[
        {'x':-0.05,
         'xanchor':"right",
         'yanchor':"top",
         'y':1.1,
         'direction':"down",
         'active':int(np.where(all_regions_iso == "USA")[0][0]),
         'showactive':True,
         "buttons": [dict(
                    label=all_regions_names[all_regions_iso == cty][0],#np.array(df_regions[df_regions.ISO == cty].name)[0],
                    method="update",
                    args=[{"y": [
                                  np.array(xr_preprocess_temp.sel(Region=cty, Ccat="C1+C2", Variable="F1g"))*1e3,
                                  np.array(xr_preprocess_temp.sel(Region=cty, Ccat="C3", Variable="F1g"))*1e3,
                                  np.array(xr_preprocess_temp.sel(Region=cty, Ccat="C6", Variable="F1g"))*1e3,
                                  np.array(xr_preprocess_temp.sel(Region=cty, Ccat="C7", Variable="F1g"))*1e3,

                                  np.array(xr_preprocess_temp.sel(Region=cty, Ccat="C1+C2", Variable="F1h"))*1e3,
                                  np.array(xr_preprocess_temp.sel(Region=cty, Ccat="C3", Variable="F1h"))*1e3,
                                  np.array(xr_preprocess_temp.sel(Region=cty, Ccat="C6", Variable="F1h"))*1e3,
                                  np.array(xr_preprocess_temp.sel(Region=cty, Ccat="C7", Variable="F1h"))*1e3,
                                  
                                  np.array(xr_preprocess_temp.sel(Region=cty, Ccat="C1+C2", Variable="F2"))*1e3,
                                  np.array(xr_preprocess_temp.sel(Region=cty, Ccat="C3", Variable="F2"))*1e3,
                                  np.array(xr_preprocess_temp.sel(Region=cty, Ccat="C6", Variable="F2"))*1e3,
                                  np.array(xr_preprocess_temp.sel(Region=cty, Ccat="C7", Variable="F2"))*1e3],
                            "text": [
                                  np.array(xr_preprocess_temp.sel(Region=cty, Ccat="C1+C2", Variable="F1g"))*1e3,
                                  np.array(xr_preprocess_temp.sel(Region=cty, Ccat="C3", Variable="F1g"))*1e3,
                                  np.array(xr_preprocess_temp.sel(Region=cty, Ccat="C6", Variable="F1g"))*1e3,
                                  np.array(xr_preprocess_temp.sel(Region=cty, Ccat="C7", Variable="F1g"))*1e3,

                                  np.array(xr_preprocess_temp.sel(Region=cty, Ccat="C1+C2", Variable="F1h"))*1e3,
                                  np.array(xr_preprocess_temp.sel(Region=cty, Ccat="C3", Variable="F1h"))*1e3,
                                  np.array(xr_preprocess_temp.sel(Region=cty, Ccat="C6", Variable="F1h"))*1e3,
                                  np.array(xr_preprocess_temp.sel(Region=cty, Ccat="C7", Variable="F1h"))*1e3,
                                  
                                  np.array(xr_preprocess_temp.sel(Region=cty, Ccat="C1+C2", Variable="F2"))*1e3,
                                  np.array(xr_preprocess_temp.sel(Region=cty, Ccat="C3", Variable="F2"))*1e3,
                                  np.array(xr_preprocess_temp.sel(Region=cty, Ccat="C6", Variable="F2"))*1e3,
                                  np.array(xr_preprocess_temp.sel(Region=cty, Ccat="C7", Variable="F2"))*1e3]
                            }],
                ) for cty_i, cty in enumerate(all_regions_iso)]
        },
    ]
)

# TITLE
fig.update_xaxes(title="Time (years)", row=1, col=2)
fig.update_yaxes(title="GHG emissions (kt CO2e)", row=1, col=1)
fig.update_yaxes(title="GHG emissions (kt CO2e)", row=1, col=2)
fig.update_yaxes(title="GHG emissions (kt CO2e)", row=1, col=3)
fig.update_layout(legend=dict(
    #yanchor="top",
    #y=-0.15,
    x=-0.1,
    y=0.9,
    xanchor="right",
    #x=0.07,
    #orientation="h"
))
fig.update_layout(height=800, autosize=True)

# ROUNDUP
fig.show()
fig_ccat_country_drop = fig
#py.plot(fig_dropdown_country, filename = 'Budgets_singlecountry_dropdown', auto_open=False, autosize=True)

#### Fig. 2 Map with countries of which NDC matches C1+C2 (dropdown: effort-sharing)

In [128]:
cs = np.zeros(shape=(len(rules), len(all_countries_iso)))
for r_i, r in enumerate(all_countries_iso):
    ndc = float((xr_total.NDC_l.sel(Time=2030, ISO=r) + xr_total.NDC_h.sel(Time=2030, ISO=r))/2)
    for es_i, es in enumerate(rules):
        budget = np.array(xr_budgets_scenario.sel(Time=2030, ISO=r, Category=categories)[es])
        if not np.isnan(ndc) and not np.isnan(budget[0]):
            wh = np.where(ndc <= budget)[0]
            if len(wh) > 0:
                cs[es_i, r_i] = int(wh[0])
            else:
                cs[es_i, r_i] = int(3)
        else:
            cs[es_i, r_i] = np.nan
dicty = {}
dicty["Region"] = all_countries_iso
dicty["C_GF"] = cs[0]
dicty["C_F1g"] = cs[1]
dicty["C_F1h"] = cs[2]
dicty["C_F2"] = cs[3]
dicty["C_F2C"] = cs[4]
df_ndc = pd.DataFrame(dicty)

eu_assessment = np.zeros(len(rules))+np.nan
eu_ndc = float((xr_total.NDC_l.sel(Time=2030, ISO='EU') + xr_total.NDC_h.sel(Time=2030, ISO='EU'))/2)
for es_i, es in enumerate(rules):
    wh = np.where(eu_ndc <= xr_budgets_scenario.sel(Time=2030, ISO='EU', Category=categories)[es])[0]
    if len(wh) > 0:
        eu_assessment[es_i] = int(wh[0])
    else:
        eu_assessment[es_i] = int(3)

In [ ]:
fig = make_subplots(rows=1, cols=1, subplot_titles=('','(a) Comparison NDCs',
                                                    '(b) Comparison Current Policies'), 
                                                    specs=[[{'type': 'choropleth'}]],#, {'type': 'choropleth'}]],
                                                    horizontal_spacing = 0.04, vertical_spacing=0.05)#, column_widths=[0.5, 0.5])#, shared_yaxes=True)

colorbar_pc = dict(len=1, x=1)
ts = ['Below 1.5 degrees', 'Well below 2 degrees', 'Below 3 degrees', 'Below 4 degrees']

fig.add_trace(
    go.Choropleth(
    locations=df_ndc['Region'],
    z = df_ndc["C_GF"],
    locationmode = 'ISO-3',
    colorscale = 'RdYlGn_r', 
    zmax = 3,
    zmin = 0,
    text = [df_ndc.iloc[c]['Region'] for c in range(len(df_ndc))],
    hovertemplate  = '%{text}',
    name="",
    marker_line_color='white', 
    marker_line_width=0.4,
    colorbar=dict(len=1, x=1, tickvals = [0, 1, 2, 3], ticktext = [categories[c]+": "+ts[c] for c in range(len(categories))], title={"text":"C-category"}),
), 1, 1)


euval = eu_assessment[rules == ['GF']][0]
fig.add_trace(
    go.Choropleth(
    locations=group_eu,
    z = np.array([euval]*len(group_eu)),
    locationmode = 'ISO-3',
    colorscale = 'RdYlGn_r',
    zmax = 3,
    zmin = 0,
    text = ["EU"]*len(group_eu),
    hovertemplate  = '%{text}',
    name="",
    marker_line_color='white', 
    marker_line_width=0.4,
    showscale=False
), 1, 1)

fig.update_geos(showocean=True, oceancolor="aliceblue")
fig.update_geos(showlakes=True, lakecolor="aliceblue")

fig.update_geos(visible=True,
                showlakes=True,
                lakecolor='rgb(255, 255, 255)',
                projection_type='natural earth',
                showcoastlines=False,)

fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction="right",
            x=0.7,
            y=-0.05,
            showactive=True,
            buttons=list(
                [
                    dict(
                        label="Grandfathering",
                        method="update",
                        args=[{"z": [df_ndc['C_GF'].astype(float), np.array([eu_assessment[rules == ['GF']][0]]*len(group_eu))],
                               "text": [[df_ndc.iloc[c]['Region'] for c in range(len(df_ndc))], ["EU"]*len(group_eu)],
                               "zmax": [3, 3],
                               "zmin": [0, 0],
                               }],
                    ),
                    # dict(
                    #     label="Per-capita",
                    #     method="update",
                    #     args=[{"z": [df_ndc['C_PC'].astype(float), df_curpol['C_PC'].astype(float)],
                    #            "text": [[df_ndc.iloc[c]['Region']+": C"+str(df_ndc.iloc[c]['C_PC']+1) for c in range(len(df_ndc))],
                    #                      [df_curpol.iloc[c]['Region']+": C"+str(df_curpol.iloc[c]['C_PC']+1) for c in range(len(df_curpol))]],
                    #            "zmax": [7, 7],
                    #            "zmin": [0, 0],
                    #            }],
                    # ),
                    # dict(
                    #     label="Per-capita Convergence",
                    #     method="update",
                    #     args=[{"z": [df_ndc['C_PCC'].astype(float), df_curpol['C_PCC'].astype(float)],
                    #            "text": [[df_ndc.iloc[c]['Region']+": C"+str(df_ndc.iloc[c]['C_PCC']+1) for c in range(len(df_ndc))],
                    #                      [df_curpol.iloc[c]['Region']+": C"+str(df_curpol.iloc[c]['C_PCC']+1) for c in range(len(df_curpol))]],
                    #            "zmax": [7, 7],
                    #            "zmin": [0, 0],
                    #            }],
                    # ),
                    dict(
                        label="Approach 1 - GDP",
                        method="update",
                        args=[{"z": [df_ndc['C_F1g'].astype(float), np.array([eu_assessment[rules == ['F1g']][0]]*len(group_eu))],
                               "text": [[df_ndc.iloc[c]['Region'] for c in range(len(df_ndc))], ["EU"]*len(group_eu)],
                               "zmax": [3, 3],
                               "zmin": [0, 0],
                               }],
                    ),
                    dict(
                        label="Approach 1 - HDI",
                        method="update",
                        args=[{"z": [df_ndc['C_F1h'].astype(float),np.array([eu_assessment[rules == ['F1h']][0]]*len(group_eu))],
                               "text": [[df_ndc.iloc[c]['Region'] for c in range(len(df_ndc))], ["EU"]*len(group_eu)],
                               "zmax": [3, 3],
                               "zmin": [0, 0],
                               }],
                    ),
                    dict(
                        label="Approach 2 - GDP",
                        method="update",
                        args=[{"z": [df_ndc['C_F2'].astype(float), np.array([eu_assessment[rules == ['F2']][0]]*len(group_eu))],
                               "text": [[df_ndc.iloc[c]['Region'] for c in range(len(df_ndc))], ["EU"]*len(group_eu)],
                               "zmax": [3, 3],
                               "zmin": [0, 0],
                               }],
                    ),
                    dict(
                        label="Approach 2 - GDP (transition)",
                        method="update",
                        args=[{"z": [df_ndc['C_F2C'].astype(float), np.array([eu_assessment[rules == ['F2C']][0]]*len(group_eu))],
                               "text": [[df_ndc.iloc[c]['Region'] for c in range(len(df_ndc))], ["EU"]*len(group_eu)],
                               "zmax": [3, 3],
                               "zmin": [0, 0],
                               }],
                    ),
                ]
            ),
        )
    ]
)
fig.update_layout(height=800, width=1500)
fig.update_layout(title_text = "Evaluating emission levels in 2030 under each effort-sharing rule. Color indicates to which temperatures respective NDCs (unconditional) correspond.")
fig.show()
fig_comparison_emissions_ndc = fig

#### Fig. 3 Emission reductions in 2030 w.r.t. 2010 (or 1990) in map or table

In [130]:
cs = np.zeros(shape=(len(rules), len(all_countries_iso)))
cs2 = np.zeros(shape=(len(rules), len(all_countries_iso)))
for r_i, r in enumerate(all_countries_iso):
    pop = float(xr_total.sel(ISO=r, Time=2030).Population)
    emis1990 = float(xr_total.sel(ISO=r, Time=1990).GHG_p)
    for es_i, es in enumerate(rules):
        cs[es_i, r_i] = xr_preprocess.sel(Region=r, Variable=es, Ccat="C1+C2")/pop*1e6
        cs2[es_i, r_i] = (xr_preprocess.sel(Region=r, Variable=es, Ccat="C1+C2")/1e3 - emis1990)/emis1990*100
dicty = {}
dicty["Region"] = all_countries_iso
dicty["C_GF"] = cs[0]
dicty["C_F1g"] = cs[1]
dicty["C_F1h"] = cs[2]
dicty["C_F2"] = cs[3]
dicty["C_F2C"] = cs[4]
df_emis = pd.DataFrame(dicty)

dicty = {}
dicty["Region"] = all_countries_iso
dicty["C_GF"] = cs2[0]
dicty["C_F1g"] = cs2[1]
dicty["C_F1h"] = cs2[2]
dicty["C_F2"] = cs2[3]
dicty["C_F2C"] = cs2[4]
df_emisr = pd.DataFrame(dicty)

In [131]:
eu_emispc = xr_budgets_scenario.sel(ISO="EU", Time=2030, Category="C1+C2")[rules] / xr_total.Population.sel(ISO='EU', Time=2030)*1e9
eu_emisch = (xr_budgets_scenario.sel(ISO="EU", Time=2030, Category="C1+C2")[rules] - float(xr_total.sel(ISO="EU", Time=1990).GHG_p)) / float(xr_total.sel(ISO="EU", Time=1990).GHG_p) *100

In [ ]:
fig = make_subplots(rows=1, cols=2, subplot_titles=('(a) Emission per capita in 2030',
                                                    '(b) Emission change of 2030 w.r.t. 1990<br>(reduction is negative)'),
                                                    specs=[[{'type': 'choropleth'}, {'type': 'choropleth'}]],
                                                    horizontal_spacing = 0.04, vertical_spacing=0.05, column_widths=[0.5, 0.5])

colorbar_abs = dict(len=1, x=1, tickvals = [0, 1, 2, 3], ticktext = ['1', '10', '100', '1000'])
colorbar_pc = dict(len=1, x=1)

fig.add_trace(
    go.Choropleth(
    locations=df_emis['Region'],
    z = df_emis["C_GF"],
    locationmode = 'ISO-3',
    colorscale = 'YlOrRd', 
    text = df_emis["C_GF"],
    zmin=0,
    zmax=14,
    hovertemplate  = '%{text}',
    name="",
    coloraxis='coloraxis',
), 1, 1)
fig.add_trace(
    go.Choropleth(
    locations=group_eu,
    z = [float(eu_emispc.GF)]*len(group_eu),
    locationmode = 'ISO-3',
    colorscale = 'YlOrRd', 
    text = [float(eu_emispc.GF)]*len(group_eu),
    zmin=0,
    zmax=14,
    hovertemplate  = '%{text}',
    name="",
    coloraxis='coloraxis',
), 1, 1)
fig.update_geos(showocean=True, oceancolor="aliceblue")
fig.update_geos(showlakes=True, lakecolor="aliceblue")

fig.update_geos(visible=True,
                showlakes=True,
                lakecolor='rgb(255, 255, 255)',
                projection_type='natural earth',
                showcoastlines=False,)

fig.add_trace(
    go.Choropleth(
    locations=df_emisr['Region'],
    z = df_emisr["C_GF"],
    locationmode = 'ISO-3',
    colorscale = 'YlOrRd', 
    text = df_emisr["C_GF"],
    hovertemplate  = '%{text}',
    name="",
    coloraxis='coloraxis2',
    showlegend=False
), 1, 2)
fig.add_trace(
    go.Choropleth(
    locations=group_eu,
    z = [float(eu_emisch.GF)]*len(group_eu),
    locationmode = 'ISO-3',
    colorscale = 'YlOrRd', 
    text = [float(eu_emisch.GF)]*len(group_eu),
    hovertemplate  = '%{text}',
    name="",
    coloraxis='coloraxis2',
    showlegend=False
), 1, 2)
fig.update_geos(showocean=True, oceancolor="aliceblue")
fig.update_geos(showlakes=True, lakecolor="aliceblue")

fig.update_geos(visible=True,
                showlakes=True,
                lakecolor='rgb(255, 255, 255)',
                projection_type='natural earth',
                showcoastlines=False,)

fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction="right",
            x=0.6,
            y=-0.05,
            showactive=True,
            buttons=list(
                [
                    dict(
                        label="Grandfathering",
                        method="update",
                        args=[{"z": [df_emis['C_GF'].astype(float), [float(eu_emispc.GF)]*len(group_eu), df_emisr['C_GF'].astype(float), [float(eu_emisch.GF)]*len(group_eu)],
                               "text": [df_emis['C_GF'].astype(float), [float(eu_emispc.GF)]*len(group_eu), df_emisr['C_GF'].astype(float), [float(eu_emisch.GF)]*len(group_eu)],
                               "zmax": [7, 7],
                               }],
                    ),
                    # dict(
                    #     label="Per-capita",
                    #     method="update",
                    #     args=[{"z": [df_emis['C_PC'].astype(float), df_emisr['C_PC'].astype(float)],
                    #            "text": [df_emis['C_PC'].astype(float), df_emisr['C_PC'].astype(float)],
                    #            "zmax": [7, 7],
                    #            }],
                    # ),
                    # dict(
                    #     label="Per-capita Convergence",
                    #     method="update",
                    #     args=[{"z": [df_emis['C_PCC'].astype(float), df_emisr['C_PCC'].astype(float)],
                    #            "text": [df_emis['C_PCC'].astype(float), df_emisr['C_PCC'].astype(float)],
                    #            "zmax": [7, 7],
                    #            }],
                    # ),
                    dict(
                        label="Approach 1 - GDP",
                        method="update",
                        args=[{"z": [df_emis['C_F1g'].astype(float), [float(eu_emispc.F1g)]*len(group_eu), df_emisr['C_F1g'].astype(float), [float(eu_emisch.F1g)]*len(group_eu)],
                               "text": [df_emis['C_F1g'].astype(float), [float(eu_emispc.F1g)]*len(group_eu), df_emisr['C_F1g'].astype(float), [float(eu_emisch.F1g)]*len(group_eu)],
                               "zmax": [7, 7],
                               }],
                    ),
                    dict(
                        label="Approach 1 - HDI",
                        method="update",
                        args=[{"z": [df_emis['C_F1h'].astype(float), [float(eu_emispc.F1h)]*len(group_eu), df_emisr['C_F1h'].astype(float), [float(eu_emisch.F1h)]*len(group_eu)],
                               "text": [df_emis['C_F1h'].astype(float), [float(eu_emispc.F1h)]*len(group_eu), df_emisr['C_F1h'].astype(float), [float(eu_emisch.F1h)]*len(group_eu)],
                               "zmax": [7, 7],
                               }],
                    ),
                    dict(
                        label="Approach 2 - GDP",
                        method="update",
                        args=[{"z": [df_emis['C_F2'].astype(float), [float(eu_emispc.F2)]*len(group_eu), df_emisr['C_F2'].astype(float), [float(eu_emisch.F2)]*len(group_eu)],
                               "text": [df_emis['C_F2'].astype(float), [float(eu_emispc.F2)]*len(group_eu), df_emisr['C_F2'].astype(float), [float(eu_emisch.F2)]*len(group_eu)],
                               "zmax": [7, 7],
                               }],
                    ),
                    dict(
                        label="Approach 2 - GDP (transition)",
                        method="update",
                        args=[{"z": [df_emis['C_F2C'].astype(float), [float(eu_emispc.F2C)]*len(group_eu), df_emisr['C_F2C'].astype(float), [float(eu_emisch.F2C)]*len(group_eu)],
                               "text": [df_emis['C_F2C'].astype(float), [float(eu_emispc.F2C)]*len(group_eu), df_emisr['C_F2C'].astype(float), [float(eu_emisch.F2C)]*len(group_eu)],
                               "zmax": [7, 7],
                               }],
                    ),
                ]
            ),
        )
    ]
)
fig.update_layout(coloraxis=dict(colorbar_title="t CO2e pc", colorscale='YlOrBr', colorbar_orientation='v', colorbar_x = -0.05, cmin=0, cmax=np.nanpercentile(df_emis['C_GF'].astype(float), 95),
                                 colorbar_thickness=23),
                  coloraxis2=dict(colorbar_title="%", colorscale='RdBu_r', colorbar_orientation='v', colorbar_x = 1.02, colorbar_thickness=23, cmin=-100, cmax=100))
fig.update_layout(height=700, width=2000)
fig.update_layout(title_text = "Emission data associated with achieving Paris (C1+C2).")
fig.show()
fig_emissions_maps = fig

#### Fig. 4 Years at which 43% reduction is reached

In [133]:
cs = np.zeros(shape=(len(rules), len(all_countries_iso)))
for r_i, r in enumerate(all_countries_iso):
    for es_i, es in enumerate(rules):
        try:
            perc = 0.57*float(xr_budgets_scenario.sel(ISO=r, Time=2020).GF.mean(dim='Category'))
            year = int(xr_budgets_scenario.Time[np.where(xr_budgets_scenario.sel(ISO=r, Category="C1+C2")[es] <= perc)[0]][0])
        except:
            year = 2101
        cs[es_i, r_i] = year
dicty = {}
dicty["Region"] = all_countries_iso
dicty["C_GF"] = cs[0]
dicty["C_F1g"] = cs[1]
dicty["C_F1h"] = cs[2]
dicty["C_F2"] = cs[3]
dicty["C_F2C"] = cs[4]
df_nz = pd.DataFrame(dicty)
median = np.median(df_nz['C_GF']) # Correct for missing data or negative current emissions
df_nz["C_GF"] = median

eu_years = np.zeros(len(rules))+np.nan
for es_i, es in enumerate(rules):
    try:
        perc = 0.57*float(xr_budgets_scenario.sel(ISO='EU', Time=2020).GF.mean(dim='Category'))
        year = int(xr_budgets_scenario.Time[np.where(xr_budgets_scenario.sel(ISO="EU", Category="C1+C2")[es] <= perc)[0]][0])
    except:
        year = 2101
    eu_years[es_i] = year

In [ ]:
fig = go.Figure()

ts = ['<1.5 (50%)', '1.5 (50%, return)', '<2 (67%)', '<2 (50%)', '<2.5 (50%)', '<3 (50%)', '<4 (50%)', '>4 (50%)']

fig.add_trace(
    go.Choropleth(
    locations=df_nz['Region'],
    z = df_nz["C_GF"],
    zmin = 2019,
    zmax = 2100,
    locationmode = 'ISO-3',
    colorscale = 'Spectral_r', 
    text = df_nz["C_GF"],
    hovertemplate  = '%{text}',
    name="",
    marker_line_color='black', 
    marker_line_width=0.4,
    colorbar=dict(len=1, x=1, title={"text":"Year"})
))
fig.add_trace(
    go.Choropleth(
    locations=group_eu,
    z = [eu_years[np.where(rules == "GF")[0][0]]]*len(group_eu),
    zmin = 2019,
    zmax = 2100,
    locationmode = 'ISO-3',
    colorscale = 'Spectral_r', 
    text = [eu_years[np.where(rules == "GF")[0][0]]]*len(group_eu),
    hovertemplate  = '%{text}',
    name="",
    marker_line_color='black', 
    marker_line_width=0.4,
    colorbar=dict(len=1, x=1, title={"text":"Year"})
))
fig.update_geos(showocean=True, oceancolor="aliceblue")
fig.update_geos(showlakes=True, lakecolor="aliceblue")

fig.update_geos(visible=True,
                showlakes=True,
                lakecolor='rgb(255, 255, 255)',
                projection_type='natural earth',
                showcoastlines=True,)

fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction="right",
            x=0.6,
            y=-0.05,
            showactive=True,
            buttons=list(
                [
                    dict(
                        label="Grandfathering",
                        method="update",
                        args=[{"z": [df_nz['C_GF'].astype(float), [eu_years[np.where(rules == "GF")[0][0]]]*len(group_eu)],
                               "text": [df_nz['C_GF'].astype(float), [eu_years[np.where(rules == "GF")[0][0]]]*len(group_eu)],
                               }],
                    ),
                    # dict(
                    #     label="Per-capita",
                    #     method="update",
                    #     args=[{"z": [df_nz['C_PC'].astype(float)],
                    #            "text": [df_nz['C_PC'].astype(float)],
                    #            }],
                    # ),
                    # dict(
                    #     label="Per-capita Convergence",
                    #     method="update",
                    #     args=[{"z": [df_nz['C_PCC'].astype(float)],
                    #            "text": [df_nz['C_PCC'].astype(float)],
                    #            }],
                    # ),
                    dict(
                        label="Approach 1 - GDP",
                        method="update",
                        args=[{"z": [df_nz['C_F1g'].astype(float), [eu_years[np.where(rules == "F1g")[0][0]]]*len(group_eu)],
                               "text": [df_nz['C_F1g'].astype(float), [eu_years[np.where(rules == "F1g")[0][0]]]*len(group_eu)],
                               }],
                    ),
                    dict(
                        label="Approach 1 - HDI",
                        method="update",
                        args=[{"z": [df_nz['C_F1h'].astype(float), [eu_years[np.where(rules == "F1h")[0][0]]]*len(group_eu)],
                               "text": [df_nz['C_F1h'].astype(float), [eu_years[np.where(rules == "F1h")[0][0]]]*len(group_eu)],
                               }],
                    ),
                    dict(
                        label="Approach 2 - GDP",
                        method="update",
                        args=[{"z": [df_nz['C_F2'].astype(float), [eu_years[np.where(rules == "F2")[0][0]]]*len(group_eu)],
                               "text": [df_nz['C_F2'].astype(float), [eu_years[np.where(rules == "F2")[0][0]]]*len(group_eu)],
                               }],
                    ),
                    dict(
                        label="Approach 2 - GDP (transition)",
                        method="update",
                        args=[{"z": [df_nz['C_F2C'].astype(float), [eu_years[np.where(rules == "F2C")[0][0]]]*len(group_eu)],
                               "text": [df_nz['C_F2C'].astype(float), [eu_years[np.where(rules == "F2C")[0][0]]]*len(group_eu)],
                               }],
                    ),
                ]
            ),
        )
    ]
)
fig.update_layout(height=800, width=1500)
fig.update_layout(title_text = "Year in which 43% of GHG emissions (compared to 2020) are reduced (in C1+C2 scenarios)")
fig.show()
fig_year = fig

#### Savings

In [135]:
figs = [fig_dropdown_country,
        fig_ccat_country_drop,
        fig_comparison_emissions_ndc,
        fig_emissions_maps,
        fig_year]

In [136]:
import os
from datetime import date
try:
    os.remove('X:/user/dekkerm/Projects/ECEMF_T5.2/Figures/Figure_fullhtml/Figure_fullhtml_'+str(hist_startyear)+'.html')
except:
    3
def html_w(typ):
    return '<html> '+typ+' <p style="font-family: Arial">'

text = ["Per-country data",
        "Now also per c-category, to check monotomy",
        "Comparison NDC and CurPol with derived emissions in 2030",
        "Emissions",
        "Year of 43% emission reduction compared to 2020. 2101 means that the country does not reach this within the coming century."]
# text = ["All computations are focused on CO2 emissions and budgets, excluding LULUCF and non-CO2. Global Carbon budget is based on Tab. SPM.2 in IPCC AR6 WGI for 2 degrees, 67% likelihood: 1150 Gt CO2 from 2020 and on. Parameter settings in all plots: period covered is 1950-2100, discount rate 1.5% and period of convergence in per-capita convergence is 30 years. All effort sharing is calculated from start 2020 and on (i.e., historical emissions are included up to and including 2019). I used two historical emission datasets, use the buttons to see where they match and where they don't. I read that PRIMAP is based on EDGAR, but I do see differences. The plots are interactive, so you can zoom and hover. Double-click to go to default layout. Scenario emissions (from AR6) involve Energy & Industrial processes, which do incorporate BECCS, but do not incorporate AFOLU.",
#         "Here the total future CCS in C1, C2 and C3 scenarios (AR6 database) is shared among all countries based on their GDP. The per capita version here is computed by dividing by cumulative population over 1995-2100 (not 1950-2100 because CCS was highly limited between 1950-1995, and the AR6 scenarios are from 1995 and on).",
#         "Here, I first computed the (discounted) historical emissions and future emissions (net). From those net emissions, I compute the positive emissions by adding CCS to it. Dividing this by the world cumulative population (past+future), the per-capita budget is distributed across the countries. A comparison between this positive budget for countries and what they already spent is presented below: 100% = historical positive budget is fully spent (>100% is overspent). (note that subtracting from this the GDP-distributed CCS yields the CVF budget per country, but this is shown in other graphs.)",
#         "The very left bars (blue) show the cumulative emissions over 2020-2100 if the country would keep emitting exactly what it does now. Later, I will put an actual baseline here.",
#         "Use the buttons to navigate through multiple effort sharing rules.",
#         "Same as fig 5, but divided by the cumulative population of 2020-2100.",
#         "I have yet to get purely CO2 data in the NDC factsheet data (Malte). For reference, I put the total GHG version in the plot, and included a recent IMAGE ENGAGE NDC run. The results for EU from the IMAGE ENGAGE run are not exactly EU: it is computed by adding WEU and CEU together, so countries like Norway, UK and Switzerland are in there (hence the elevated level of the grey line)."]

# fig.write_image("Figures/Paperfigures/Figure_1"+sub+".pdf")
# fig.write_image("Figures/Paperfigures/Figure_1"+sub+".png", scale=3)
today = date.today()
td = today.strftime("%B %d, %Y")

with open('X:/user/dekkerm/Projects/ECEMF_T5.2/Figures/Figure_fullhtml/Figure_fullhtml_'+str(hist_startyear)+'.html', 'a') as f:
    f.write(html_w('<h1>')+'Effort Sharing figures</p></h1>')
    f.write(html_w('<body>')+'Version: '+td+'</p></body>')

    for n_i in range(len(figs)):
        if n_i > 0:
            f.write('<hr>')
        f.write(html_w('<h1>')+'Fig. '+str(n_i+1)+'</p></h1>')
        f.write(html_w('<body>')+text[n_i]+'</p></body>')
        f.write(figs[n_i].to_html(full_html=False, include_plotlyjs='cdn'))